In [ ]:
import os
import sys
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.path.abspath("")))
from utils.lmdb_utils import  *
import utils.FPHA_utils as FPHA
import utils.YOLO_utils as YOLO

In [ ]:
train_file_name, test_file_name, train_xyz_gt, test_xyz_gt \
 = FPHA.get_train_test_pairs('color', FPHA.DIR)

all_file_name = train_file_name + test_file_name
all_xyz_gt = np.concatenate((train_xyz_gt, test_xyz_gt))
all_uvd_gt = FPHA.xyz2uvd_color(all_xyz_gt)

In [ ]:
index0 = list(np.unique(np.argwhere(all_uvd_gt[..., 0] > FPHA.ORI_WIDTH*2)[:, 0]))
index1 = list(np.unique(np.argwhere(all_uvd_gt[..., 0] < -500)[:, 0]))
index2 = list(np.unique(np.argwhere(all_uvd_gt[..., 1] > FPHA.ORI_HEIGHT*2)[:, 0]))
index3 = list(np.unique(np.argwhere(all_uvd_gt[..., 1] < -500)[:, 0]))

all_index = np.unique(index0 + index1 + index2 + index3)
print(len(all_index))

In [ ]:
# Visualize
# from IPython.display import clear_output
# for idx in range(len(all_index)):
#     index = all_index[idx]
#     print(idx, index)
#     print(all_file_name[index])
#     img = Image.open(os.path.join(FPHA.DIR, 'Video_files', all_file_name[index]))
#     fig, ax = plt.subplots(figsize=(5,5))
#     ax.imshow(img)
#     FPHA.visualize_joints_2d(ax, all_uvd_gt[index][FPHA.REORDER_IDX], joint_idxs=False)
#     plt.show()
#     input("Press Enter to continue...")

#     clear_output()

In [ ]:
index = 93973
img = Image.open(os.path.join(FPHA.DIR, 'Video_files', all_file_name[index]))
fig, ax = plt.subplots(figsize=(15,15))
FPHA.visualize_joints_2d(ax, all_uvd_gt[index][FPHA.REORDER_IDX], joint_idxs=False)
ax.imshow(img)

In [ ]:
# Save to file
def get_idx(key):
    return all_file_name.index(key)

def get_all_idx(subject_action_seq):
    all_idx = []
    for i, fn in enumerate(all_file_name):
        if subject_action_seq in fn:
            all_idx.append(i)
    return all_idx

custom_add_bad = get_all_idx('Subject_1/unfold_glasses/5') + \
                get_all_idx('Subject_4/read_letter/3') + \
                get_all_idx('Subject_1/unfold_glasses/5')
                
# custom add
for bad_idx in custom_add_bad:
    if bad_idx not in all_index:
        all_index = np.append(all_index, bad_idx)

print("WRITING BAD IMGS TO FILE")
with open(os.path.join(FPHA.DIR, 'bad_imgs.txt'), 'w') as f:
    for index in tqdm(all_index):
        f.write("%s\n" % all_file_name[index])